In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [2]:
mapping = pd.read_excel("company_master_mapping.xlsx")
# price_data = d.fetch_price_data(no_of_years=20)
price_data = pd.read_csv('stockPriceData.csv')
# Convert 'Date' to datetime type
price_data['Date'] = pd.to_datetime(price_data['Date'])

# Create a dictionary mapping from SYMBOL_NSE to SYMBOL_CM
mapping = dict(zip(mapping["SYMBOL_NSE"], mapping["SYMBOL_CM"]))
price_data["Symbol"] = price_data["Symbol"].replace(mapping)

top_500 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
# Filter price data for dates after '2006-01-01' and drop duplicates
price_data.drop_duplicates(['Date', 'Symbol'], inplace=True)

#company master swapping
# company_master = d.fetch_data_from_database(table_name='Companymaster')
company_master = pd.read_csv('Companymaster_3_2_2025.csv')
company_master["SYMBOL"] = company_master["SYMBOL"].replace(mapping)

yearly = pd.read_csv('Finance_pl_3_2_2025.csv')
Quaterly = pd.read_csv('Quarterly_3_2_2025.csv')
Finance_bs =  pd.read_csv('Finance_bs_3_2_2025.csv')[[ 'Year_end', 'Fincode', 'Share_Capital', 'Reserve']]

rebal_dates = pd.read_csv('GrowthDate.csv')

rebal_dates["Date"] = pd.to_datetime(rebal_dates["Date"])
rebal=rebal_dates['Date']

Rebalance_Qtr=list(rebal_dates['Quarter'])
Rebalance_Date=list(rebal_dates['Date'])
Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))
date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))
top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)
index_constituents_data= top_500

X = Quaterly[Quaterly['Result_Type'] == 'Q']
print(X.columns)
X
# # yearly.to_csv('yearly.csv')
# # Quaterly.to_csv('Quaterly.csv')
# # Finance_bs.to_csv('Finance_bs.csv')
# # company_master.to_csv('Companymaster.csv')

Index(['Fincode', 'Result_Type', 'NoOfMonths', 'Date_End', 'NET_SALES',
       'INT_ADV', 'INC_INV', 'INT_BAL', 'INT_OTHERS', 'OTHER_INCOME',
       ...
       'exc_foreign_exch_gain', 'exc_other', 'Curr_tax', 'Def_tax',
       'Fringe_benefits', 'Prior_Period_Tax', 'Mat_Credit', 'Other_Tax',
       'flag', 'notes'],
      dtype='object', length=132)


,Fincode,Result_Type,NoOfMonths,Date_End,NET_SALES,INT_ADV,INC_INV,INT_BAL,INT_OTHERS,OTHER_INCOME,...,exc_foreign_exch_gain,exc_other,Curr_tax,Def_tax,Fringe_benefits,Prior_Period_Tax,Mat_Credit,Other_Tax,flag,notes
96,100002,Q,3.0,200003,1491.10,0.0,0.0,0.0,0.0,25.90,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,A,NaN
97,100002,Q,3.0,200006,1648.37,0.0,0.0,0.0,0.0,49.47,...,0.0,0.00,8.00,0.00,0.0,0.0,0.0,0.0,A,NaN
98,100002,Q,3.0,200009,1935.69,0.0,0.0,0.0,0.0,11.27,...,0.0,0.00,62.00,0.00,0.0,0.0,0.0,0.0,A,NaN
99,100002,Q,3.0,200012,2860.59,0.0,0.0,0.0,0.0,48.83,...,0.0,0.00,95.00,0.00,0.0,0.0,0.0,0.0,A,NaN
100,100002,Q,3.0,200103,2021.72,0.0,0.0,0.0,0.0,18.45,...,0.0,0.00,13.00,0.00,0.0,0.0,0.0,0.0,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925157,317142,Q,3.0,202409,510.91,0.0,0.0,0.0,0.0,2.60,...,0.0,0.00,0.00,0.15,0.0,0.0,0.0,0.0,A,NaN
925159,317635,Q,3.0,202412,13920.09,0.0,0.0,0.0,0.0,166.66,...,0.0,0.00,477.10,-32.14,0.0,0.0,0.0,0.0,A,NaN
925163,318188,Q,3.0,202409,679.80,0.0,0.0,0.0,0.0,5.84,...,0.0,-9.54,0.00,0.00,0.0,0.0,0.0,0.0,A,NaN
925164,318188,Q,3.0,202412,3367.49,0.0,0.0,0.0,0.0,50.14,...,0.0,-1005.23,0.00,0.00,0.0,0.0,0.0,0.0,A,NaN


In [3]:
# company_master = pd.read_csv('Companymaster.csv')
# yearly = pd.read_csv('yearly.csv')
# Quaterly = pd.read_csv('Quaterly.csv')
# Finance_bs = pd.read_csv('Finance_bs.csv')
# price_data = pd.read_csv('price_data.csv')
# top_500 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))

# rebal_dates = pd.read_csv('./RebalanceDate.csv')
# rebal_dates["Date"] = pd.to_datetime(rebal_dates["Date"])
# rebal=rebal_dates['Date']

# Rebalance_Qtr=list(rebal_dates['Quarter'])
# Rebalance_Date=list(rebal_dates['Date'])
# Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))/
# date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))
# top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)
# index_constituents_data= top_500

# X = Quaterly[Quaterly['Result_Type'] == 'Q']
# print(X.columns)
# X.drop(columns='Unnamed: 0', inplace=True)

In [4]:
# Calculating Networth 
Finance_bs['NetWorth'] = Finance_bs['Share_Capital'] + Finance_bs['Reserve']
Finance_bs2 = Finance_bs[['Year_end', 'Fincode', 'NetWorth']]
Finance_bs3 = pd.merge(Finance_bs2, company_master[['FINCODE', 'SYMBOL']], left_on='Fincode', right_on='FINCODE')
Finance_bs3 = Finance_bs3[['Year_end', 'FINCODE', 'SYMBOL', 'NetWorth']]

In [5]:
# Revisd PAT in units
pat_df= X[['Date_End', 'Fincode', 'PAT']]
pat_df['PAT'] = pat_df['PAT']/10

In [6]:
# Calculating Networth on quarterly basis
p =pd.merge(pat_df, Finance_bs3, left_on=['Date_End','Fincode'], right_on=['Year_end','FINCODE'], how='left')
p['SYMBOL'] = p.groupby('Fincode', group_keys=False)['SYMBOL'].apply(lambda x: x.fillna(method='ffill'))
p = p.drop(columns=['Year_end', 'FINCODE'])

In [7]:
# Quarterly Networth Calculation
def fill_networth(row, prev_networth):
    if pd.isna(row['NetWorth']):
        return prev_networth + row['PAT']
    else:
        return row['NetWorth']

def fill_networth_column(df):
    df = df.sort_values(by=['Fincode', 'Date_End']) 
    df['NetWorth_Filled'] = np.nan
    
    for fincode in df['Fincode'].unique():
        prev_networth = None
        for i, row in df[df['Fincode'] == fincode].iterrows():
            if prev_networth is None:
                prev_networth = row['NetWorth']
            else:
                df.at[i, 'NetWorth_Filled'] = fill_networth(row, prev_networth)
                prev_networth = df.at[i, 'NetWorth_Filled']
    
    return df

# Apply the function
p = fill_networth_column(p)

# Fill any remaining NaN in the original NetWorth column with the filled values
p['NetWorth'] = p['NetWorth'].combine_first(p['NetWorth_Filled'])
p.drop(columns=['NetWorth_Filled'], inplace=True)
p = p.dropna()

In [8]:
# Calculating ROE from scratch
p['ROE'] = p.groupby('SYMBOL', group_keys=False).apply(lambda x: x['PAT']/x['NetWorth'])
p['ROE_ttm'] = p.groupby('SYMBOL', group_keys=False)['ROE'].apply(lambda x: x.rolling(window=4).sum())
roe_ttm = p[['Fincode', 'Date_End', 'ROE_ttm']]

In [9]:
# p.to_csv('ROEdata.csv')

In [10]:
# opm_df = X[['Fincode', 'Date_End', 'OPERATING_PROFIT', 'NET_SALES']]

# opm_df2 = pd.merge(opm_df, company_master[['FINCODE', 'SYMBOL']], left_on='Fincode', right_on='FINCODE')
# opm_df2['OPM'] = opm_df2.groupby('SYMBOL', group_keys=False).apply(lambda x: x['OPERATING_PROFIT'].div(x['NET_SALES']))
# opm_df2['5Y_OPM_mean'] = opm_df2.groupby('SYMBOL')['OPM'].transform(lambda x : x.rolling(20).mean())
# opm_df3 = opm_df2[['Date_End', 'Fincode', '5Y_OPM_mean']].dropna()
# opm_df3

In [11]:
# gpm_df = X[['Fincode', 'Date_End', 'GROSS_PROFIT', 'NET_SALES','NIM']]
# gpm_df2 = pd.merge(gpm_df, company_master[['FINCODE', 'SYMBOL']], left_on='Fincode', right_on='FINCODE')
# gpm_df2['GPM'] = gpm_df2.groupby('SYMBOL', group_keys=False).apply(lambda x: x['GROSS_PROFIT'].div(x['NET_SALES']))
# gpm_df2['GPM'] = gpm_df2['GPM']*100
# #   GPM rowth Calculation
# def calculate_yoy_GPM_growth(GPM):
#     """Calculates YoY EPS Growth based on the provided rules."""
#     growth = []
#     for i in range(len(GPM)):
#         if i < 4:
#             growth.append(np.nan) 
#         else:
#             prev_eps = GPM.iloc[i - 4]  
#             curr_eps = GPM.iloc[i]      
#             if prev_eps > 0:
#                 growth.append((curr_eps - prev_eps) / prev_eps)
#             elif prev_eps < 0:
#                 growth.append(-(curr_eps - prev_eps) / prev_eps)
#             else:
#                 growth.append(np.nan)  
#     return pd.Series(growth, index=GPM.index)  

# # Calculating YoY EPS Growth for each quarter
# gpm_df2['YoY_GPM'] = gpm_df2.groupby('SYMBOL')['GPM'].transform(calculate_yoy_GPM_growth)

# #   NIM rowth Calculation
# def calculate_yoy_NIM_growth(NIM):
#     """Calculates YoY EPS Growth based on the provided rules."""
#     growth = []
#     for i in range(len(NIM)):
#         if i < 4:
#             growth.append(np.nan) 
#         else:
#             prev_eps = NIM.iloc[i - 4]  
#             curr_eps = NIM.iloc[i]      
#             if prev_eps > 0:
#                 growth.append((curr_eps - prev_eps) / prev_eps)
#             elif prev_eps < 0:
#                 growth.append(-(curr_eps - prev_eps) / prev_eps)
#             else:
#                 growth.append(np.nan)  
#     return pd.Series(growth, index=NIM.index)  

# # Calculating YoY EPS Growth for each quarter
# gpm_df2['YoY_NIM'] = gpm_df2.groupby('SYMBOL')['NIM'].transform(calculate_yoy_NIM_growth)
# gpm_df2 = gpm_df2[['Date_End', 'Fincode', 'YoY_GPM','YoY_NIM']]
# gpm_df2

In [12]:
quality = X[['Fincode','Date_End','Debt/Equity Ratio', 'Adj_eps_abs']]
quality = pd.merge(quality, roe_ttm, on=['Fincode', 'Date_End'])
pl_quality = pd.merge(quality, company_master, how='inner', left_on='Fincode', right_on='FINCODE')
# pl_quality2 = pd.merge(pl_quality, opm_df3, on=['Fincode', 'Date_End'], how='inner')
# # print(pl_quality2.columns)
# pl_quality3 = pd.merge(pl_quality2, gpm_df2, on=['Fincode', 'Date_End'], how='inner')

In [13]:
a = pl_quality[['Fincode', 'Date_End', 'Debt/Equity Ratio', 'ROE_ttm', 'SYMBOL', 'Adj_eps_abs']].reset_index(drop=True)

# Removing the rows with NaN Symbol Names
a = a[a.SYMBOL.notna()]

# Filtering Stocks which have been historically in top 500 universe only.
b = a[a.SYMBOL.isin(top_500.Symbol.unique())]

# Filter out stocks with fewer than 4 quarters of history
symbol_quarter_counts = b.groupby('SYMBOL').size()
valid_symbols = symbol_quarter_counts[symbol_quarter_counts >= 4].index
b = b[b['SYMBOL'].isin(valid_symbols)]

# Define a function to check for 2 consecutive quarters of negative EPS within a 1-year period
def has_2_consecutive_negative_in_last_1_year(series):
    return series.rolling(window=4, min_periods=4).apply(
        lambda x: any((x[i] < 0 and x[i + 1] < 0) for i in range(len(x) - 1)), raw=True
    )

# Apply the function to create a flag for exclusion
b['exclude_flag'] = b.groupby('SYMBOL')['Adj_eps_abs'].transform(has_2_consecutive_negative_in_last_1_year)

# Filter out rows based on the exclude flag
b_filtered = b[b['exclude_flag'] != 1].reset_index(drop=True)

# Drop the intermediate column used for filtering
b_filtered = b_filtered.drop(columns=['exclude_flag'])

# Calculate Adjusted EPS TTM
b_filtered['Adj_eps_abs_TTM'] = b_filtered.groupby('SYMBOL')['Adj_eps_abs'].transform(lambda x: x.rolling(4).sum())

In [14]:
# a = pl_quality[['Fincode', 'Date_End', 'Debt/Equity Ratio', 'ROE_ttm', 'SYMBOL', 'Adj_eps_abs']].reset_index(drop=True)

# # Removing the rows with NaN Symbol Names
# a = a[a.SYMBOL.notna()]

# # Filtering Stocks which have been historically in top 500 universe only.
# b = a[a.SYMBOL.isin(top_500.Symbol.unique())]

# # Calculate Adjusted EPS TTM
# b['Adj_eps_abs_TTM'] = b.groupby('SYMBOL')['Adj_eps_abs'].transform(lambda x: x.rolling(4).sum())

# # Define a function to check for negatives in the last 16 quarters
# def has_negative_in_last_4_years(series):
#     return series.rolling(window=16, min_periods=16).apply(lambda x: (x < 0).any(), raw=True)

# # Apply the function to create a flag for exclusion
# b['exclude_flag'] = b.groupby('SYMBOL')['Adj_eps_abs_TTM'].transform(has_negative_in_last_4_years)

# # Remove symbols with fewer than 16 quarters of data
# # Count the number of quarters for each symbol
# symbol_quarter_counts = b.groupby('SYMBOL').size()

# # Create a list of symbols with at least 16 quarters
# valid_symbols = symbol_quarter_counts[symbol_quarter_counts >= 16].index

# # Filter the DataFrame to keep only rows with valid symbols
# b = b[b['SYMBOL'].isin(valid_symbols)]

# # Filter out rows based on the exclude flag
# b_filtered = b[b['exclude_flag'] != 1].reset_index(drop=True)

# # Drop the intermediate column used for filtering
# b_filtered = b_filtered.drop(columns=['exclude_flag'])

# Mapping GICS to each Symbol
# SectorThemeGICS = d.fetch_data_from_database(table_name='SectorThemeGICS', no_of_years=25)

SectorThemeGICS = pd.read_excel('SectorMapping.xlsx')
df = pd.merge(b_filtered, SectorThemeGICS[['Symbol', 'Sector']], left_on='SYMBOL', right_on='Symbol', how='left')

# Calculating ROE for TTM for the past 3 Years years
df['ROE_ttm'] = df.groupby('Symbol')['ROE_ttm'].transform(lambda x : x.rolling(12).mean())

# Create a new column for quarter information
df['Quarter'] = pd.to_datetime(df['Date_End'], format='%Y%m').dt.quarter

#  EPS Growth Calculation
def calculate_yoy_eps_growth(eps):
    """Calculates YoY EPS Growth based on the provided rules."""
    growth = []
    for i in range(len(eps)):
        if i < 4:
            growth.append(np.nan) 
        else:
            prev_eps = eps.iloc[i - 4]  
            curr_eps = eps.iloc[i]      
            if prev_eps > 0:
                growth.append((curr_eps - prev_eps) / prev_eps)
            elif prev_eps < 0:
                growth.append(-(curr_eps - prev_eps) / prev_eps)
            else:
                growth.append(np.nan)  
    return pd.Series(growth, index=eps.index)  

# Calculating YoY EPS Growth for each quarter
df['YoY_EPS_Growth'] = df.groupby('SYMBOL')['Adj_eps_abs'].transform(calculate_yoy_eps_growth)

In [15]:
# Calculate 3-year mean and std deviation for each quarter separately
def calc_quarterly_stats(group):
    group = group.sort_values('Date_End')
    group['Mean_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=3, min_periods=1).mean()
    group['Std_YoY_EPS_Growth'] = group['YoY_EPS_Growth'].rolling(window=3, min_periods=1).std()
    return group

# Apply function to each SYMBOL and Quarter group
df = df.groupby(['SYMBOL', 'Quarter'], group_keys=False).apply(calc_quarterly_stats)

# Dropping Quarter column after calculation to keep the original dataframe structure
df.drop(columns=['Quarter'], inplace=True)

# Calculating rolling 3-year mean and std deviation for EPS growth
df['Mean_YoY_EPS_Growth'] = df.groupby('SYMBOL')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=12,min_periods=4).mean())
df['Std_YoY_EPS_Growth'] = df.groupby('SYMBOL')['YoY_EPS_Growth'].transform(lambda x: x.rolling(window=12,min_periods=4).std())

# Calulating EPS Growth Variability
df['EPS_Growth_Variability'] = df['Mean_YoY_EPS_Growth'].div(df['Std_YoY_EPS_Growth'])

In [16]:
# # Calculate 5-year mean and std deviation for each quarter separately
# def calc_quarterly_stats(group):
#     group = group.sort_values('Date_End')
#     group['Mean_YoY_GPM_Growth'] = group['YoY_GPM'].rolling(window=5, min_periods=1).mean()
#     group['Std_YoY_GPM_Growth'] = group['YoY_GPM'].rolling(window=5, min_periods=1).std()
#     return group

# # Apply function to each SYMBOL and Quarter group
# df = df.groupby(['SYMBOL', 'Quarter'], group_keys=False).apply(calc_quarterly_stats)
# # Dropping Quarter column after calculation to keep the original dataframe structure
# # df.drop(columns=['Quarter'], inplace=True)

# # Calculating rolling 5-year mean and std deviation for EPS growth
# df['Mean_YoY_GPM_Growth'] = df.groupby('SYMBOL')['YoY_GPM'].transform(lambda x: x.rolling(window=20).mean())
# df['Std_YoY_GPM_Growth'] = df.groupby('SYMBOL')['YoY_GPM'].transform(lambda x: x.rolling(window=20).std())

# # Calulating EPS Growth Variability
# df['GPM_Growth_Variability'] = df['Mean_YoY_GPM_Growth'].div(df['Std_YoY_GPM_Growth'])

In [17]:
# # Calculate 5-year mean and std deviation for each quarter separately
# def calc_quarterly_stats(group):
#     group = group.sort_values('Date_End')
#     group['Mean_YoY_NIM_Growth'] = group['YoY_NIM'].rolling(window=5, min_periods=1).mean()
#     group['Std_YoY_NIM_Growth'] = group['YoY_NIM'].rolling(window=5, min_periods=1).std()
#     return group

# # Apply function to each SYMBOL and Quarter group
# df = df.groupby(['SYMBOL', 'Quarter'], group_keys=False).apply(calc_quarterly_stats)
# # Dropping Quarter column after calculation to keep the original dataframe structure
# df.drop(columns=['Quarter'], inplace=True)

# # Calculating rolling 5-year mean and std deviation for EPS growth
# df['Mean_YoY_NIM_Growth'] = df.groupby('SYMBOL')['YoY_NIM'].transform(lambda x: x.rolling(window=20).mean())
# df['Std_YoY_NIM_Growth'] = df.groupby('SYMBOL')['YoY_NIM'].transform(lambda x: x.rolling(window=20).std())

# # Calulating EPS Growth Variability
# df['NIM_Growth_Variability'] = df['Mean_YoY_NIM_Growth'].div(df['Std_YoY_NIM_Growth'])

In [18]:
# Split data into three sectors: Bank, Finance, and Others
bank_df = df[df['Sector'] == 'Bank']
finance_df = df[df['Sector'] == 'Finance']
non_financials_df = df[~df['Sector'].isin(['Bank', 'Finance'])]

# Calculate metrics for Bank sector
bank_df['mean_roe_bank'] = bank_df.groupby('Date_End')['ROE_ttm'].transform('mean')
bank_df['std_roe_bank'] = bank_df.groupby('Date_End')['ROE_ttm'].transform('std')
bank_df['mean_eps_growth_variability_bank'] = bank_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
bank_df['std_eps_growth_variability_bank'] = bank_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

# Calculate metrics for Finance sector
finance_df['mean_roe_fin'] = finance_df.groupby('Date_End')['ROE_ttm'].transform('mean')
finance_df['std_roe_fin'] = finance_df.groupby('Date_End')['ROE_ttm'].transform('std')
finance_df['mean_eps_growth_variability_fin'] = finance_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
finance_df['std_eps_growth_variability_fin'] = finance_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

# Calculate metrics for non-financials
non_financials_df['mean_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('mean')
non_financials_df['std_roe'] = non_financials_df.groupby('Date_End')['ROE_ttm'].transform('std')
non_financials_df['mean_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('mean')
non_financials_df['std_de'] = non_financials_df.groupby('Date_End')['Debt/Equity Ratio'].transform('std')
non_financials_df['mean_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('mean')
non_financials_df['std_eps_growth_variability'] = non_financials_df.groupby('Date_End')['EPS_Growth_Variability'].transform('std')

# Z-Score calculations for Bank sector
bank_df['Z_ROE_ttm_bank'] = (bank_df['ROE_ttm'] - bank_df['mean_roe_bank']) / bank_df['std_roe_bank']
bank_df['Z_EPS_Growth_Variability_bank'] = (bank_df['EPS_Growth_Variability'] - bank_df['mean_eps_growth_variability_bank']) / bank_df['std_eps_growth_variability_bank']

# Z-Score calculations for Finance sector
finance_df['Z_ROE_ttm_fin'] = (finance_df['ROE_ttm'] - finance_df['mean_roe_fin']) / finance_df['std_roe_fin']
finance_df['Z_EPS_Growth_Variability_fin'] = (finance_df['EPS_Growth_Variability'] - finance_df['mean_eps_growth_variability_fin']) / finance_df['std_eps_growth_variability_fin']

# Z-Score calculations for non-financials
non_financials_df['Z_ROE_ttm'] = (non_financials_df['ROE_ttm'] - non_financials_df['mean_roe']) / non_financials_df['std_roe']
non_financials_df['Z_DE'] = (non_financials_df['Debt/Equity Ratio'] - non_financials_df['mean_de']) / non_financials_df['std_de']
non_financials_df['Z_EPS_Growth_Variability'] = (non_financials_df['EPS_Growth_Variability'] - non_financials_df['mean_eps_growth_variability']) / non_financials_df['std_eps_growth_variability']

# Weighted Average Z Quality Score calculation
def calculate_weighted_avg_z(row):
    if row['Sector'] == 'Bank':
        return (1/2) * row['Z_ROE_ttm_bank'] - (1/2) * abs(row['Z_EPS_Growth_Variability_bank'])
    elif row['Sector'] == 'Finance':
        return (1/2) * row['Z_ROE_ttm_fin'] - (1/2) * abs(row['Z_EPS_Growth_Variability_fin'])
    else:
        return (1/3) * row['Z_ROE_ttm'] - (1/3) * abs(row['Z_DE']) - (1/3) * abs(row['Z_EPS_Growth_Variability'])

# Combine all dataframes
df_combined = pd.concat([bank_df, finance_df, non_financials_df]).reset_index(drop=True)

# Calculate weighted average Z-score
df_combined['WeightedAvgZ'] = df_combined.apply(calculate_weighted_avg_z, axis=1)

# Calculate Normalized Quality Score
df_combined['Quality_Score'] = np.where(df_combined['WeightedAvgZ'] >= 0, 
                                      1 + df_combined['WeightedAvgZ'], 
                                      (1 - df_combined['WeightedAvgZ'])**-1)

# Final dataframe with required columns
df = df_combined[['Date_End', 'Symbol', 'Sector', 'Quality_Score']].dropna().reset_index(drop=True)

In [19]:
# GrowthDate = d.fetch_data_from_database(table_name='GrowthDate', no_of_years=25)[['Date', 'Qtr']]
# GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')
# final_df =  pd.merge(df, GrowthDate, left_on='Date_End', right_on='Qtr').drop(columns=['Date_End', 'Qtr'])
# final_df = final_df[['Date', 'Symbol', 'Sector', 'Quality_Score']].sort_values(by='Date').reset_index(drop=True)
# price_data['Date'] = pd.to_datetime(price_data['Date'])
# price_data.drop(columns='Unnamed: 0', inplace=True)
# merged_df = pd.merge(final_df[['Date', 'Symbol', 'Quality_Score']], price_data, on=['Date', 'Symbol'], how='outer')
# merged_df['Quality_Score'] = merged_df.groupby('Symbol', group_keys=False)['Quality_Score'].apply(lambda x: x.fillna(method='ffill'))
# merged_df
# merged_df.tail(500).dropna().sort_values(by='Quality_pct_rank', ascending=False)

In [20]:
# df['Quality_pct_rank'] = df.groupby('Date_End', group_keys=False)['Quality_Score'].apply(lambda x : x.rank(pct=True))
# GrowthDate = d.fetch_data_from_database(table_name='GrowthDate', no_of_years=25)[['Date', 'Qtr']]
GrowthDate = pd.read_csv('GrowthDate.csv')[['Date', 'Qtr']]
GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')

In [21]:
# GrowthDate Mapping  
final_df =  pd.merge(df, GrowthDate, left_on='Date_End', right_on='Qtr').drop(columns=['Date_End', 'Qtr'])
final_df = final_df[['Date', 'Symbol', 'Sector', 'Quality_Score']].sort_values(by='Date').reset_index(drop=True)
final_df

,Date,Symbol,Sector,Quality_Score
0,2006-05-31,BANKINDIA,Bank,0.703128
1,2006-05-31,J&KBANK,Bank,0.688327
2,2006-05-31,BAJAJHLDNG,Finance,0.740377
3,2006-05-31,IDBI,Bank,0.652245
4,2006-05-31,PNB,Bank,0.866487
...,...,...,...,...
47526,2024-11-18,BRIGADE,Realty,0.805791
47527,2024-11-18,KOKUYOCMLN,FMCG,0.518851
47528,2024-11-18,SECURKLOUD,IT,0.757954
47529,2024-11-18,TATACHEM,Chemicals,0.813175


In [22]:
# Merging Quality Ranks with price data universe
price_data['Date'] = pd.to_datetime(price_data['Date'])
final_df['Date'] = pd.to_datetime(final_df['Date'])
merged_df = pd.merge(final_df[['Date', 'Symbol', 'Quality_Score']], price_data, on=['Date', 'Symbol'], how='outer')
merged_df =  merged_df.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
merged_df['Quality_pct_rank'] = merged_df.groupby('Date', group_keys=False)['Quality_Score'].apply(lambda x : x.rank(pct=True))

In [23]:
merged_df[['Quality_pct_rank', 'Quality_Score']] = merged_df.groupby('Symbol', group_keys=False)[['Quality_pct_rank', 'Quality_Score']].apply(lambda x: x.fillna(method='ffill'))
qualityfinal = merged_df[['Date','Symbol','Quality_Score','Quality_pct_rank']]
qualityfinal

,Date,Symbol,Quality_Score,Quality_pct_rank
7,1995-05-02,RELIANCE,NaN,NaN
9,1995-05-02,TATASTEEL,NaN,NaN
5,1995-05-02,HINDPETRO,NaN,NaN
3,1995-05-02,GRASIM,NaN,NaN
1,1995-05-02,BPCL,NaN,NaN
...,...,...,...,...
7479537,2025-04-28,ARVIND,1.011440,0.685714
7480988,2025-04-28,RAILTEL,0.523958,0.086294
7480611,2025-04-28,MARKSANS,0.721118,0.403553
7480807,2025-04-28,OLECTRA,0.697492,0.337563


In [24]:
top500 = qualityfinal.filter(["Symbol", "Date","Quality_Score"])

## Convert the long from Dataframe to wide form dataframe
top500 = top500.pivot_table(index='Date',columns='Symbol',values='Quality_Score').reset_index()
## Shifting the Date column
top500["Date"] = top500["Date"].shift(-1)
## Fill NaN value of date with todays date.
top500.iloc[-1, top500.columns.get_loc("Date")] = pd.to_datetime(date.today())

## Re-converting the wide form dataframe to long form dataframe
top500 = top500.melt(id_vars='Date', value_name = "Quality_Score")
## Drop na and reset the index
top500 = top500.dropna().reset_index(drop = True)
top500 = top500[top500["Date"] >= "2006-01-01"].reset_index(drop = True).copy()

In [25]:
# merged_df['Quality_Score'] = merged_df.groupby('Symbol', group_keys=False)['Quality_Score'].apply(lambda x: x.fillna(method='ffill'))
# merged_df.dropna(inplace=True)
# merged_df = merged_df[['Date','Symbol','Quality_Score']]
# merged_df.to_csv('Akshat_Quality.csv')

In [28]:
#Not dropped NaNs
top500.to_csv('iii.csv')

In [ ]:
# rebalanceFrequency=2
# from rebalancedate import *
# op=rebalancedates(price_data,'2006-06-02',17, rebalanceFrequency)
# rebalanceDates=op
# df_rebalance_dates = pd.DataFrame({'Timestamp': pd.to_datetime(rebalanceDates)})
# df_rebalance_dates.rename(columns={'Timestamp': 'Date'}, inplace=True)
# qualityfinal['Date'] = pd.to_datetime(qualityfinal['Date'])
# qualityfinal = qualityfinal[qualityfinal['Date'].isin(df_rebalance_dates['Date'])].reset_index(drop=True)
# top_stocks = 25
# df_stocks = qualityfinal.copy()
# df_stocks = df_stocks.groupby('Date').apply(lambda x: x.nlargest(top_stocks, 'Quality_Score')).reset_index(drop=True)
# df_stocks
qualf = qualityfinal.dropna()
qualf = qualf.sort_values(by=['Date','Quality_Score'], ascending=[True,False])
top_25_stocks_each_day = qualf.groupby('Date').tail(25)
top_25_stocks_each_day['Date'] = pd.to_datetime(top_25_stocks_each_day['Date'])
rebalanceFrequency=2
from rebalancedate import *
op=rebalancedates(price_data,'2006-06-02',17, rebalanceFrequency)
rebalanceDates=op[1]
df_rebalance_dates = pd.DataFrame({'Date': pd.to_datetime(rebalanceDates)})
df_rebalance_dates
top_25_stocks_each_day = top_25_stocks_each_day[top_25_stocks_each_day['Date'].isin(df_rebalance_dates['Date'])].reset_index(drop=True)
top_25_stocks_each_day

,Date,Symbol,Quality_Score,Quality_pct_rank
0,2008-10-20,IDBI,0.805616,0.694444
1,2008-10-20,J&KBANK,0.804224,0.666667
2,2008-10-20,INGVYSYABK,0.789333,0.638889
3,2008-10-20,ORIENTBANK,0.788121,0.611111
4,2008-10-20,AXISBANK,0.782608,0.583333
...,...,...,...,...
2445,2024-12-18,GODREJIND,0.293667,0.007772
2446,2024-12-18,JSWSTEEL,0.279599,0.010152
2447,2024-12-18,FLUOROCHEM,0.272366,0.007614
2448,2024-12-18,RAMCOCEM,0.260914,0.005076


In [36]:
top_25_stocks_each_day.to_csv('QualityShort.csv', index=False)

In [24]:
qualityfinal[qualityfinal['Date'] == '2025-01-17'].sort_values(by='Quality_Score', ascending=False).head(30)

,Date,Symbol,Quality_Score,Quality_pct_rank
2621187,2025-01-17,HDFCAMC,2.286558,1.000000
1369533,2025-01-17,COALINDIA,2.190064,0.997462
4413235,2025-01-17,NAM-INDIA,2.100873,0.994924
5714654,2025-01-17,SBICARD,1.936810,0.992386
5356595,2025-01-17,RECLTD,1.901405,0.989848
4930449,2025-01-17,PFC,1.754050,0.987310
750442,2025-01-17,BAJFINANCE,1.729338,0.984772
54989,2025-01-17,360ONE,1.700138,0.982234
1277076,2025-01-17,CGPOWER,1.670161,0.979695
1452768,2025-01-17,CREDITACC,1.655558,0.977157


In [83]:
# Filter for the specific date
data_all = qualityfinal[qualityfinal['Date'] == '2024-12-06']

# Filter after dropping NaN values
data_no_na = qualityfinal[qualityfinal['Date'] == '2024-12-06'].dropna()

# Companies present before dropping NaNs
companies_with_na = data_all['Symbol']

# Companies remaining after dropping NaNs
companies_without_na = data_no_na['Symbol']

# Identify companies dropped due to NaNs
dropped_due_to_na = companies_with_na[~companies_with_na.isin(companies_without_na)]


In [84]:
# Filter for the specific date
data_all = qualityfinal[qualityfinal['Date'] == '2024-12-06']

# Filter after dropping NaN values
data_no_na = qualityfinal[qualityfinal['Date'] == '2024-12-06'].dropna()

# Companies present before dropping NaNs
companies_with_na = data_all['Symbol']

# Companies remaining after dropping NaNs
companies_without_na = data_no_na['Symbol']

# Identify companies dropped due to NaNs
dropped_due_to_na = companies_with_na[~companies_with_na.isin(companies_without_na)]

# Save results into a DataFrame
results_df = pd.DataFrame({
    'With_NaN': companies_with_na.tolist(),
    'Without_NaN': companies_without_na.tolist() + [None] * (len(companies_with_na) - len(companies_without_na)),
    'Dropped_Due_To_NaN': dropped_due_to_na.tolist() + [None] * (len(companies_with_na) - len(dropped_due_to_na))
})

# Display the results DataFrame
print(results_df)

# Optionally, save to a CSV file
results_df.to_csv('dropped_companies_due_to_na.csv', index=False)


       With_NaN Without_NaN Dropped_Due_To_NaN
0      RELIANCE    RELIANCE               LICI
1           TCS         TCS             ZOMATO
2      HDFCBANK    HDFCBANK             JIOFIN
3    BHARTIARTL  BHARTIARTL         ADANIGREEN
4     ICICIBANK   ICICIBANK            HYUNDAI
..          ...         ...                ...
495    SAPPHIRE        None               None
496    INDIACEM        None               None
497         SCI        None               None
498     KIRLPNU        None               None
499  SANOFICONR        None               None

[500 rows x 3 columns]


In [111]:
churnDf = df_stocks.groupby("Date")["Symbol"].apply(set).rename("Current").reset_index()
churnDf["Previous"] = churnDf["Current"].shift()
churnDf = churnDf.dropna()
churnDf["Churn"] = churnDf.apply(lambda x : len(x["Current"] - x["Previous"])/len(x["Previous"]), axis = 1)
churnDf["Churn"].mean()

0.3222222222222222

In [112]:
df_stocks.to_csv('Quality2.csv', index=False)

0                   IT
1          Diversified
2              Finance
3                   IT
4       Infrastructure
             ...      
2111    Infrastructure
2112         Chemicals
2113        Healthcare
2114       Hospitality
2115                IT
Name: Sector, Length: 2116, dtype: object